In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
# from sklearn.metrics import accuracy_score, mean_squared_error, f1_score, log_loss, r2_score


# load in March Madness data, target is for regression, target2 is for classification
df = pd.read_csv('MarchMadnessData2024.csv')
df['team_1_win'] = (df['margin'] > 0).astype(int)
df.head()

,Win-Loss Percentage_team1,SRS_team1,SOS_team1,Team Points_team1,Opponent Ponts_team1,Minutes Played_team1,FG_team1,FGA_team1,FG%_team1,3P_team1,...,AST_team2,STL_team2,BLK_team2,TOV_team2,PF_team2,margin,team1_location_Away,team1_location_Home,team1_location_Neutral,team_1_win
0,0.690,5.94,-2.10,2283.0,2015.0,1170.0,790.0,1854.0,0.426,269.0,...,498.0,207.0,100.0,339.0,495.0,-5.0,True,False,False,0
1,0.643,-2.40,-3.28,2002.0,1858.0,1125.0,737.0,1689.0,0.436,201.0,...,498.0,207.0,100.0,339.0,495.0,7.0,False,True,False,1
2,0.778,3.39,-3.81,2256.0,1992.0,1085.0,819.0,1754.0,0.467,295.0,...,313.0,198.0,76.0,248.0,503.0,19.0,False,True,False,1
3,0.778,3.39,-3.81,2256.0,1992.0,1085.0,819.0,1754.0,0.467,295.0,...,407.0,231.0,95.0,349.0,461.0,13.0,False,True,False,1
4,0.778,3.39,-3.81,2256.0,1992.0,1085.0,819.0,1754.0,0.467,295.0,...,418.0,143.0,91.0,253.0,403.0,-16.0,False,False,True,0


In [15]:
features = ['Win-Loss Percentage_team1', 'Win-Loss Percentage_team2', 'SRS_team1', 'SRS_team2', 'SOS_team1', 'SOS_team2',
            'FG%_team1', 'FG%_team2', '3P%_team1', '3P%_team2', 'TOV_team1', 'TOV_team2', 'ORB_team1', 'ORB_team2']

y = df['team_1_win']
X = df[features]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [16]:
import tensorflow as tf
from tensorflow import keras

# better NN?
def build_tcn():
    model = keras.Sequential([
        keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='causal', input_shape=(X_train_scaled.shape[1], 1)),
        keras.layers.BatchNormalization(),
        keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='causal'),
        keras.layers.BatchNormalization(),
        keras.layers.GlobalAveragePooling1D(),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')  # Output probability
    ])
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Reshape data for Conv1D
X_train_tcn = np.expand_dims(X_train_scaled, axis=-1)
X_test_tcn = np.expand_dims(X_test_scaled, axis=-1)

# Train the TCN model
tcn_model = build_tcn()
tcn_model.fit(X_train_tcn, y_train, epochs=50, batch_size=32, validation_data=(X_test_tcn, y_test), verbose=1)

# Predict probabilities
# tcn_probs = tcn_model.predict(X_test_tcn).flatten()

Epoch 1/50


C:\Users\cdpea\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 36ms/step - accuracy: 0.6634 - loss: 0.6320 - val_accuracy: 0.6834 - val_loss: 0.6498
Epoch 2/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7040 - loss: 0.5683 - val_accuracy: 0.6834 - val_loss: 0.6225
Epoch 3/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7174 - loss: 0.5509 - val_accuracy: 0.6834 - val_loss: 0.6254
Epoch 4/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7157 - loss: 0.5369 - val_accuracy: 0.6834 - val_loss: 0.6371
Epoch 5/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7233 - loss: 0.5202 - val_accuracy: 0.6834 - val_loss: 0.6445
Epoch 6/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7224 - loss: 0.5213 - val_accuracy: 0.6834 - val_loss: 0.6577
Epoch 7/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7123 - loss: 0.5320 - val_accuracy: 0.6834 - val_loss: 0.6568
Epoch 8/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 0.6968 - loss: 0.5092 - val_accuracy: 0.6834 - val_loss: 0.6659
E

In [ ]:
import numpy as np
import pandas as pd

# Load March Madness Data Dump
data_dump = pd.read_csv('march_madness_data_dump.csv')

# Initial Matchups
team1 = [
    'Auburn', 'Marquette', 'Clemson', 'Arizona', 'Illinois', 'Kentucky', 'Missouri', 'Michigan St', 'Florida', 'Creighton',
    'BYU', 'Michigan', 'UCLA', 'Texas Tech', 'Gonzaga', 'St Johns', 'Duke', 'Connecticut', 'Oregon', 'Maryland', 'Mississippi',
    'Iowa St', 'Kansas', 'Alabama', 'Houston', 'Memphis', 'Purdue', 'Texas A&M', 'Louisville', 'Wisconsin', 'St Marys',
    'Tennessee'
]

team2 = [
    'Jackson St', 'Baylor', 'Colorado St', 'High Point', 'Drake', 'Troy', 'W Virginia', 'Wofford', 'Norfolk St', 'New Mexico',
    'Xavier', 'Akron', 'UCSD', 'Utah Valley', 'Oklahoma', 'Neb Omaha', 'American', 'Miss State', 'Liberty', 'Yale', 'VCU',
    'NC-Wilmgton', 'Utah St', 'Bryant', 'SIU Edward', 'Georgia', 'McNeese St', 'Lipscomb', 'San Diego St', 'Rob Morris', 'Arkansas',
    'Montana'
]
print(len(team1))
print(len(team2))

# Merge data dump with game data
df = pd.DataFrame({'team1': team1, 'team2': team2})

merge1 = pd.merge(df, data_dump, left_on='team1', right_on='School', how='left')
final = pd.merge(merge1, data_dump, left_on='team2', right_on='School', how='left', suffixes=('_team1', '_team2'))

print(final)

# print(final.isnull().sum())  # Shows count of NaNs per column
# print(final[final.isnull().any(axis=1)])  # Prints full rows with NaNs

# One-hot encode 'team1_location' and drop unnecessary columns (adjust as needed)
# final = pd.get_dummies(final, columns=['team1_location'])
# final = final.drop(columns=['team1', 'team2', 'Rk_team1', 'Rk_team2', 'School_team1', 'School_team2'])
final = final[features]

# Normalize data
# scaler = MinMaxScaler()
final_norm = scaler.fit_transform(final)


32
32
          team1         team2  Rk_team1 School_team1  \
0        Auburn    Jackson St        17       Auburn   
1     Marquette        Baylor       162    Marquette   
2       Clemson   Colorado St        53      Clemson   
3       Iowa St    High Point       127      Iowa St   
4      Illinois         Drake       119     Illinois   
5      Kentucky          Troy       138     Kentucky   
6      Missouri    W Virginia       183     Missouri   
7   Michigan St       Wofford       176  Michigan St   
8       Florida    Norfolk St        87      Florida   
9     Creighton    New Mexico        64    Creighton   
10          BYU        Xavier        30          BYU   
11       Purdue      Lipscomb       243       Purdue   
12         UCLA          UCSD       324         UCLA   
13   Texas Tech   Utah Valley       310   Texas Tech   
14      Gonzaga      Oklahoma       104      Gonzaga   
15     St Johns     Neb Omaha       289     St Johns   
16         Duke      American        75   

In [ ]:
# Simulate matchups using TCN model probabilities
def get_win_probability(X_game, model):
    """Returns win probability of team1 beating team2"""
    X_game = X_game.reshape(1, -1, 1)  # Reshape for TCN model
    return model.predict(X_game)[0, 0]  # Probability of team1 winning

# Monte Carlo Bracket Simulation
def simulate_brackets(model, team1, team2, final_norm, num_simulations=100):
    """Simulates the tournament using model probabilities"""
    tournament_wins = {team: 0 for team in set(team1 + team2)}

    for _ in range(num_simulations):
        round_teams_1, round_teams_2 = team1.copy(), team2.copy()

        while len(round_teams_1) > 1:  # Loop through rounds
            next_round_teams = []
            for i in range(len(round_teams_1)):
                team_A, team_B = round_teams_1[i], round_teams_2[i]

                # Get row index for current matchup
                row_index = i  # Since final_norm is ordered, this matches the game order
                X_game = final_norm[row_index]  # Extract game features

                # Get probability and pick a winner
                win_prob_A = get_win_probability(X_game, model)
                print("Probability of ", team_A, " beating ", team_B, " is ", win_prob_A)
                winner = np.random.choice([team_A, team_B], p=[win_prob_A, 1 - win_prob_A])

                next_round_teams.append(winner)

                # Track championship wins
                if len(next_round_teams) == 1:
                    tournament_wins[winner] += 1

            # Prepare for the next round
            round_teams_1, round_teams_2 = next_round_teams[::2], next_round_teams[1::2]

    return tournament_wins

# Run Monte Carlo Simulations
bracket_simulations = simulate_brackets(tcn_model, team1, team2, final_norm)

# Find most likely champions
best_teams = sorted(bracket_simulations.items(), key=lambda x: x[1], reverse=True)
print("Most likely champions:", best_teams[:10])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Probability of  Auburn  beating  Jackson St  is  0.99989015
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Probability of  Marquette  beating  Baylor  is  0.9707151
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Probability of  Clemson  beating  Colorado St  is  0.75404865
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Probability of  Iowa St  beating  High Point  is  0.8694328
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Probability of  Illinois  beating  Drake  is  0.97989637
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Probability of  Kentucky  beating  Troy  is  0.9517626
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Probability of  Missouri  beating  W Virginia  is  0.94609356
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Probability of  Michigan St  beating  Wofford  is  0.75383747
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Probability of  Florida  beating  Norfolk St  is  0.70762396
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Probability of  Creighton  beating  New Mexico  is  0.9872833
1/1 ━━━━━━━━━━━━━━━━━

KeyboardInterrupt: 

In [28]:
def generate_optimized_brackets(tcn_model, team1, team2, final_norm, num_brackets=5):
    """Generates multiple optimized brackets using model-predicted probabilities on a per-game basis."""
    optimized_brackets = []
    
    for _ in range(num_brackets):
        bracket = []
        round_teams_1, round_teams_2 = team1.copy(), team2.copy()

        # Store original matchups
        current_round = list(zip(round_teams_1, round_teams_2))
        bracket.append(current_round)

        while len(round_teams_1) > 1:
            next_round_teams = []
            current_round = []
            
            for i in range(len(round_teams_1)):
                team_A, team_B = round_teams_1[i], round_teams_2[i]

                # Extract game data for the matchup (NumPy indexing)
                game_data = final_norm[i]

                # Get probability of team_A winning
                win_prob_A = get_win_probability(game_data, tcn_model)

                # Ensure probabilities are valid
                win_prob_A = np.clip(win_prob_A, 0.01, 0.99)

                # Simulate winner using the probability
                winner = np.random.choice([team_A, team_B], p=[win_prob_A, 1 - win_prob_A])

                next_round_teams.append(winner)
                current_round.append((team_A, team_B, winner))  # Store matchup with winner

            bracket.append(current_round)
            round_teams_1, round_teams_2 = next_round_teams[::2], next_round_teams[1::2]

        optimized_brackets.append(bracket)

    return optimized_brackets

# Generate 5 optimized brackets
best_brackets = generate_optimized_brackets(tcn_model, team1, team2, final_norm, num_brackets=5)

# Print results
for i, bracket in enumerate(best_brackets):
    print(f"\n========== Bracket {i+1} ==========")
    for round_num, round_matchups in enumerate(bracket):
        print(f"\n🏀 Round {round_num + 1}:")
        for matchup in round_matchups:
            if len(matchup) == 2:
                print(f"  {matchup[0]} vs {matchup[1]}")
            else:
                print(f"  {matchup[0]} vs {matchup[1]} → Winner: {matchup[2]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━